<a href="https://colab.research.google.com/github/ariepratama/data-augmentation-coling2020/blob/main/Training_language_model_mixed_with_pos_tags_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets stanza

     |████████████████████████████████| 4.7 MB 7.9 MB/s 
     |████████████████████████████████| 365 kB 48.2 MB/s 
     |████████████████████████████████| 574 kB 54.8 MB/s 
     |████████████████████████████████| 596 kB 43.8 MB/s 
     |████████████████████████████████| 101 kB 8.2 MB/s 
     |████████████████████████████████| 6.6 MB 36.8 MB/s 
     |████████████████████████████████| 115 kB 49.9 MB/s 
     |████████████████████████████████| 212 kB 52.9 MB/s 
     |████████████████████████████████| 141 kB 54.0 MB/s 
     |████████████████████████████████| 127 kB 53.6 MB/s 
     |████████████████████████████████| 197 kB 58.4 MB/s 


In [ ]:
!pip list | grep stanza

stanza                        1.4.0


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import Trainer
from transformers import TrainingArguments
from transformers import DataCollatorForLanguageModeling
from transformers import BertModel, BertTokenizer, BertForMaskedLM
from datasets import load_dataset
import stanza
import numpy as np
from nltk import Tree, ParentedTree
from typing import *
from transformers.trainer import *
from transformers import DefaultDataCollator
from transformers import EarlyStoppingCallback

In [ ]:
NLP = stanza.Pipeline(lang="en", processors="tokenize,pos,constituency", tokenize_pretokenized=True)

2022-08-08 01:32:33 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-08-08 01:32:33 INFO: Use device: gpu
2022-08-08 01:32:33 INFO: Loading: tokenize
2022-08-08 01:32:33 INFO: Loading: pos
2022-08-08 01:32:38 INFO: Loading: constituency
2022-08-08 01:32:40 INFO: Done loading processors!


In [ ]:
model_name = "allenai/scibert_scivocab_cased"
model = BertForMaskedLM.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# last training data is empty|
train_dataset = load_dataset('conll2003', split='train[:-1]')

Reusing dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


In [ ]:
val_dataset = load_dataset('conll2003', split='validation')

Reusing dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


In [ ]:
print(f"training dataset length: {len(train_dataset)}")
print(f"validation dataset length: {len(val_dataset)}")

training dataset length: 14040
validation dataset length: 3250


In [ ]:
postoi = {'"': 0, "''": 1, '#': 2, '$': 3, '(': 4, ')': 5, ',': 6, '.': 7, ':': 8, '``': 9, 'CC': 10, 'CD': 11, 'DT': 12,
 'EX': 13, 'FW': 14, 'IN': 15, 'JJ': 16, 'JJR': 17, 'JJS': 18, 'LS': 19, 'MD': 20, 'NN': 21, 'NNP': 22, 'NNPS': 23,
 'NNS': 24, 'NN|SYM': 25, 'PDT': 26, 'POS': 27, 'PRP': 28, 'PRP$': 29, 'RB': 30, 'RBR': 31, 'RBS': 32, 'RP': 33,
 'SYM': 34, 'TO': 35, 'UH': 36, 'VB': 37, 'VBD': 38, 'VBG': 39, 'VBN': 40, 'VBP': 41, 'VBZ': 42, 'WDT': 43,
 'WP': 44, 'WP$': 45, 'WRB': 46}

# itopos = {v: k for k,v in postoi.items()}
# taglist = ['LS', 'TO', 'VBN', "''", 'WP', 'UH', 'VBG', 'JJ', 'VBZ', '--', 'VBP', 'NN', 'DT', 'PRP', ':', 'WP$', 'NNPS', 'PRP$', 'WDT', '(', ')', '.', ',', '``', '$', 'RB', 'RBR', 'RBS', 'VBD', 'IN', 'FW', 'RP', 'JJR', 'JJS', 'PDT', 'MD', 'VB', 'WRB', 'NNP', 'EX', 'NNS', 'SYM', 'CC', 'CD', 'POS']
# postoi = {v: i for i, v in enumerate(taglist)}
itopos = {v: k for k,v in postoi.items()}

In [ ]:
tokenizer.add_tokens([pos.lower() for pos in postoi.keys()])
tokenizer.add_tokens([pos.upper() for pos in postoi.keys()])
model.resize_token_embeddings(len(tokenizer) * 2)

Embedding(62284, 768)

In [ ]:
def tag_tokens(sentences: Union[List[Text], List[List[Text]]], nlp=NLP, lower_tag: bool = True) -> Union[List[Tuple[Text, Text]], List[List[Tuple[Text, Text]]]]:
  """convert list of tokens to tuple of [tag, token]
  """
  results = []
  document_texts = [" ".join(sentence) for sentence in sentences]
  in_docs = [stanza.Document([], text=document_text) for document_text in document_texts]
  stanza_docs = NLP(in_docs)

  for doc in stanza_docs:
    tag_token_result = []
    for preterminal in doc.sentences[0].constituency.yield_preterminals():
      tag = preterminal.label
      if lower_tag:
        tag = tag.lower()
        
      token = preterminal.children[0].label
      tag_token_result.append((tag, token))
    results.append(tag_token_result)
  return results


In [ ]:
def random_replace(sentences: List[List[Tuple[Text, Text]]], p=0.2) -> List[List[Text]]:
  results = []
  for sentence in sentences:
    final_text = []
    for tag, token in sentence:
      coin_toss_head = np.random.binomial(1, 0.5)
      new_token = token
      if coin_toss_head:
        new_token = tag

      final_text.append(new_token)
    results.append(final_text)
  return results

In [ ]:

def augment_and_mask(sentences: List[List[Text]], posset: Set[Text], 
                     mask_token: Text, limit=10) -> List[List[Text]]:
  """ Produced 1 -> n sentences, for each sentence, 
      automatically generate masked token on each token that deemed as pos tag / 
      consituent tag
  """
  resulting_sentences = []
  for sentence in sentences:
    potential_token_ids = set()

    for token_id, token in enumerate(sentence):
      if token in posset:
        potential_token_ids.add(token_id)
    
    for token_id in potential_token_ids:
      augmented_sentence = [token for token in sentence]
      augmented_sentence[token_id] = mask_token
      resulting_sentences.append(augmented_sentence)

  return resulting_sentences


In [ ]:
posset = set(postoi.keys()).union(set([k.lower() for k in  postoi.keys()]))

def preprocess(datum, replace_to_pos_prob=0.2, posset=posset):
    """
    1. tag tokens with constiuent
    2. randomly replace token with the constiuent
    """
    tagged_tokens = tag_tokens(datum["tokens"])
    final_tokens = random_replace(tagged_tokens)
    
    augmented_sentences = [
        augment_and_mask([sentence], posset, tokenizer.mask_token)
        for sentence in final_tokens
    ]
    flattened_augmented_sentences = [
        sentence
        for doc in augmented_sentences
        for sentence in doc
    ]
    
    
    tokenized = tokenizer(flattened_augmented_sentences, padding=False, is_split_into_words=True)
    del tokenized["token_type_ids"]
    
    return tokenized

In [ ]:
encoded_train_dataset = train_dataset.map(
    preprocess, 
    batched=True, 
    batch_size=500,
    remove_columns=["id", "tokens", "pos_tags", "chunk_tags", "ner_tags"]
)

  0%|          | 0/29 [00:00<?, ?ba/s]

In [ ]:
tokenizer.decode(
    [y for x in encoded_train_dataset["input_ids"][:15] for y in x]
).split("[SEP]")

['[CLS] [MASK] rejects jj nn to boycott jj nn. ',
 ' [CLS] nnp rejects [MASK] nn to boycott jj nn. ',
 ' [CLS] nnp rejects jj [MASK] to boycott jj nn. ',
 ' [CLS] nnp rejects jj nn [MASK] boycott jj nn. ',
 ' [CLS] nnp rejects jj nn to boycott [MASK] nn. ',
 ' [CLS] nnp rejects jj nn to boycott jj [MASK]. ',
 ' [CLS] nnp rejects jj nn to boycott jj nn [MASK] ',
 ' [CLS] brussels [MASK] ',
 ' [CLS] the [MASK] commission said on nnp prp disagreed in jj advice to consumers to shun jj lamb in nns determine whether mad cow nn md be transmitted in sheep. ',
 ' [CLS] the jj commission said on [MASK] prp disagreed in jj advice to consumers to shun jj lamb in nns determine whether mad cow nn md be transmitted in sheep. ',
 ' [CLS] the jj commission said on nnp [MASK] disagreed in jj advice to consumers to shun jj lamb in nns determine whether mad cow nn md be transmitted in sheep. ',
 ' [CLS] the jj commission said on nnp prp disagreed [MASK] jj advice to consumers to shun jj lamb in nns determ

In [ ]:
def group_text(samples, chunk_size: int):
    keys = list(samples.keys())
    # concatenate per batch
    concatenated_samples = {
        k: sum(samples[k], [])
        for k in keys
    }
    
    total_length = len(concatenated_samples[keys[0]])
    # dropping last remainder of chunk
    total_length = (total_length // chunk_size) * chunk_size
    result = {
        key: [
            value[chunk_start_index: chunk_start_index + chunk_size]
            for chunk_start_index in range(0, total_length, chunk_size)
        ]
        for key, value in concatenated_samples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
chunk_size = 500
grouped_train_dataset = encoded_train_dataset.map(
    lambda data: group_text(data, chunk_size=chunk_size),
    batched=True, 
    batch_size=1000
)

  0%|          | 0/114 [00:00<?, ?ba/s]

In [ ]:
tokenizer.decode(grouped_train_dataset["input_ids"][0]).split("[SEP]")

['[CLS] [MASK] rejects jj nn to boycott jj nn. ',
 ' [CLS] nnp rejects [MASK] nn to boycott jj nn. ',
 ' [CLS] nnp rejects jj [MASK] to boycott jj nn. ',
 ' [CLS] nnp rejects jj nn [MASK] boycott jj nn. ',
 ' [CLS] nnp rejects jj nn to boycott [MASK] nn. ',
 ' [CLS] nnp rejects jj nn to boycott jj [MASK]. ',
 ' [CLS] nnp rejects jj nn to boycott jj nn [MASK] ',
 ' [CLS] brussels [MASK] ',
 ' [CLS] the [MASK] commission said on nnp prp disagreed in jj advice to consumers to shun jj lamb in nns determine whether mad cow nn md be transmitted in sheep. ',
 ' [CLS] the jj commission said on [MASK] prp disagreed in jj advice to consumers to shun jj lamb in nns determine whether mad cow nn md be transmitted in sheep. ',
 ' [CLS] the jj commission said on nnp [MASK] disagreed in jj advice to consumers to shun jj lamb in nns determine whether mad cow nn md be transmitted in sheep. ',
 ' [CLS] the jj commission said on nnp prp disagreed [MASK] jj advice to consumers to shun jj lamb in nns determ

In [ ]:
def preprocess_val(datum):
  tagged_tokens = tag_tokens(datum["tokens"])
  final_tokens = random_replace(tagged_tokens)
  
  tokenized = tokenizer(final_tokens, padding=False, is_split_into_words=True)
  del tokenized["token_type_ids"]
    
  return tokenized

In [ ]:
grouped_val_dataset = train_dataset.map(
    preprocess_val,
    batched=True,
    batch_size=500,
    remove_columns=["id", "tokens", "pos_tags", "chunk_tags", "ner_tags"]
).map(
    lambda data: group_text(data, chunk_size=chunk_size),
    batched=True, 
    batch_size=1000
)

  0%|          | 0/29 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

In [ ]:
#  prevent to use other dataset as validation
# grouped_val_dataset = val_dataset.map(
#     preprocess_val,
#     batched=True,
#     batch_size=500,
#     remove_columns=["id", "tokens", "pos_tags", "chunk_tags", "ner_tags"]
# ).map(
#     lambda data: group_text(data, chunk_size=chunk_size),
#     batched=True, 
#     batch_size=1000
# )

In [ ]:
tokenizer.decode(grouped_val_dataset["input_ids"][0]).split("[SEP]")

['[CLS] nnp rejects german nn to vb jj lamb. ',
 ' [CLS] peter blackburn ',
 ' [CLS] nnp cd ',
 ' [CLS] dt european commission vbd in thursday prp vbd in german advice in nns to shun jj lamb in nns determine whether jj cow disease md be transmitted to sheep. ',
 " [CLS] nnp's representative in dt european nnp pos veterinary nn werner nnp said on wednesday consumers md buy nn from countries jj than nnp in the jj advice was jjr. ",
 " [CLS] `` we don't vb any such nn because we don't see any nns in it, '' the commission pos chief spokesman nnp nnp der pas vbd dt news nn. ",
 ' [CLS] he vbd further scientific study vbd required cc if it vbd found that nn was needed prp md vb taken in the nnp nnp. ',
 ' [CLS] prp said a proposal last nn by nnp nnp nnp franz nnp to ban sheep brains, nns and jj cords in the human cc nn nn nns vbd dt highly specific cc jj move to protect human health. ',
 ' [CLS] nnp proposed eu - wide measures after reports from nnp cc france that in laboratory conditions nn

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

ValueError: ignored

In [ ]:
# sample training (orig): [SEP] [CLS] Senator kelly of Arizona bought the world one
# sample training (tag mixture): [SEP] [CLS] nnp kelly of nnp vbd dt world one
# sample training (datum 1): [SEP] [CLS] [MASK] kelly of nnp vbd dt world one -> Senator kelly of nnp vbd dt world one
# sample training (datum 2): [SEP] [CLS] nnp kelly of [MASK] vbd dt world one -> nnp kelly of Arizona vbd dt world one
# sample training (datum 3): [SEP] [CLS] nnp kelly of nnp [MASK] dt world one -> nnp kelly of nnp bought dt world one
# sample eval    : [SEP] [CLS] [MASK] kelly of [MASK] vbd dt world one
# constraint the data_collator to only mask the POS tags: eval?

In [ ]:
batch_size = 8
logging_steps = len(grouped_train_dataset) // batch_size

training_args = TrainingArguments(
    output_dir="/tmp/scibert-lm-v2-finetuned-20",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=20,
    learning_rate=2e-5,
    weight_decay=1e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=logging_steps,
    # label_smoothing_factor=0.1,
    push_to_hub=True,
    fp16=True,
    load_best_model_at_end=True,
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer, 
    mlm_probability=0.15
)

In [ ]:
default_data_collator = DefaultDataCollator()

In [ ]:
# will need to override Trainer.get_train_dataloader
# and remove the collate_fn function for training, since we already collate it in the beginning

class CustomTrainer(Trainer):
  def get_train_dataloader(self) -> DataLoader:
      if self.train_dataset is None:
        raise ValueError("Trainer: training requires a train_dataset.")

      train_dataset = self.train_dataset
      train_dataset = self._remove_unused_columns(train_dataset, description="training")
      train_sampler = self._get_train_sampler()

      return DataLoader(
          train_dataset,
          batch_size=self.args.train_batch_size,
          sampler=train_sampler,
          # collate_fn=self.data_collator, we don't use data collator in this training
          collate_fn=default_data_collator,
          drop_last=self.args.dataloader_drop_last,
          num_workers=self.args.dataloader_num_workers,
          pin_memory=self.args.dataloader_pin_memory,
      )

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=grouped_train_dataset,
    eval_dataset=grouped_val_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


/tmp/scibert-lm-v2-finetuned-20 is already a clone of https://huggingface.co/ariesutiono/scibert-lm-v2-finetuned-20. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend


In [ ]:
trainer

NameError: ignored

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7018
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 17560


Epoch,Training Loss,Validation Loss
1,0.021100,15.197062
2,0.000100,16.877384
3,0.000100,15.795233


***** Running Evaluation *****
  Num examples = 571
  Batch size = 8
Saving model checkpoint to /tmp/scibert-lm-v2-finetuned-20/checkpoint-878
Configuration saved in /tmp/scibert-lm-v2-finetuned-20/checkpoint-878/config.json
Model weights saved in /tmp/scibert-lm-v2-finetuned-20/checkpoint-878/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 571
  Batch size = 8
Saving model checkpoint to /tmp/scibert-lm-v2-finetuned-20/checkpoint-1756
Configuration saved in /tmp/scibert-lm-v2-finetuned-20/checkpoint-1756/config.json
Model weights saved in /tmp/scibert-lm-v2-finetuned-20/checkpoint-1756/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 571
  Batch size = 8
Saving model checkpoint to /tmp/scibert-lm-v2-finetuned-20/checkpoint-2634
Configuration saved in /tmp/scibert-lm-v2-finetuned-20/checkpoint-2634/config.json
Model weights saved in /tmp/scibert-lm-v2-finetuned-20/checkpoint-2634/pytorch_model.bin


Training completed. Do not forget to share your model

TrainOutput(global_step=2634, training_loss=0.007087271895372034, metrics={'train_runtime': 1804.7771, 'train_samples_per_second': 77.771, 'train_steps_per_second': 9.73, 'total_flos': 5413641042744000.0, 'train_loss': 0.007087271895372034, 'epoch': 3.0})

In [ ]:
tokenizer.save_pretrained("/tmp/scibert-lm-v2-finetuned-20")

tokenizer config file saved in /tmp/scibert-lm-v2-finetuned-20/tokenizer_config.json
Special tokens file saved in /tmp/scibert-lm-v2-finetuned-20/special_tokens_map.json
added tokens file saved in /tmp/scibert-lm-v2-finetuned-20/added_tokens.json


('/tmp/scibert-lm-v2-finetuned-20/tokenizer_config.json',
 '/tmp/scibert-lm-v2-finetuned-20/special_tokens_map.json',
 '/tmp/scibert-lm-v2-finetuned-20/vocab.txt',
 '/tmp/scibert-lm-v2-finetuned-20/added_tokens.json')

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to /tmp/scibert-lm-v2-finetuned-20
Configuration saved in /tmp/scibert-lm-v2-finetuned-20/config.json
Model weights saved in /tmp/scibert-lm-v2-finetuned-20/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Aug08_01-43-03_3f560333a24e/events.out.tfevents.1659922988.3f560333a24e.483.0:  65%|######5  …

To https://huggingface.co/ariesutiono/scibert-lm-v2-finetuned-20
   4a11015..1be1019  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}, 'dataset': {'name': 'conll2003', 'type': 'conll2003', 'config': 'conll2003', 'split': 'train[:-1]', 'args': 'conll2003'}}
To https://huggingface.co/ariesutiono/scibert-lm-v2-finetuned-20
   1be1019..af88569  main -> main



'https://huggingface.co/ariesutiono/scibert-lm-v2-finetuned-20/commit/1be1019e6e84b16f38e3ed8eaae5d0ff14397069'